In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geolite2 import  geolite2
import seaborn as sns

In [2]:
feb9 = pd.read_csv('/Users/sa24/Capstone/The-CyberChase/Raw Data/2023-02-09.csv')